In [2]:
import dask.dataframe as dd

# Load data with all columns as objects first
movielens_movies_df = dd.read_csv('MovieLens/movie.csv', dtype='object')
movielens_ratings_df = dd.read_csv('MovieLens/rating.csv', dtype='object')
imdb_movies_df = dd.read_csv('IMDB/title.basics.tsv', sep='\t', dtype='object')
imdb_ratings_df = dd.read_csv('IMDB/title.ratings.tsv', sep='\t', dtype='object')
tmdb_movies_df = dd.read_csv('TMDB/tmdb_6000_movie_dataset.csv', dtype='object')
tmdb_ratings_df = dd.read_csv('TMDB/tmdb_6000_movie_ratings.csv', dtype='object')
netflix_ratings_df = dd.read_csv('Netflix_Ratings/Netflix_User_Ratings.csv', dtype='object')
rotten_movies_df = dd.read_csv('Rotten_Tomatoes/rotten_tomatoes_movies.csv', dtype='object')
rotten_reviews_df = dd.read_csv('Rotten_Tomatoes/rotten_tomatoes_movie_reviews.csv', dtype='object')

# Convert necessary columns to appropriate types after cleaning
movielens_ratings_df['userId'] = dd.to_numeric(movielens_ratings_df['userId'], errors='coerce')
movielens_ratings_df['movieId'] = dd.to_numeric(movielens_ratings_df['movieId'], errors='coerce')
movielens_ratings_df['rating'] = dd.to_numeric(movielens_ratings_df['rating'], errors='coerce')
movielens_ratings_df['timestamp'] = dd.to_datetime(movielens_ratings_df['timestamp'], errors='coerce')

imdb_movies_df['startYear'] = dd.to_numeric(imdb_movies_df['startYear'], errors='coerce')
imdb_movies_df['endYear'] = dd.to_numeric(imdb_movies_df['endYear'], errors='coerce')
imdb_movies_df['runtimeMinutes'] = dd.to_numeric(imdb_movies_df['runtimeMinutes'], errors='coerce')

imdb_ratings_df['averageRating'] = dd.to_numeric(imdb_ratings_df['averageRating'], errors='coerce')
imdb_ratings_df['numVotes'] = dd.to_numeric(imdb_ratings_df['numVotes'], errors='coerce')

tmdb_ratings_df['userId'] = dd.to_numeric(tmdb_ratings_df['userId'], errors='coerce')
tmdb_ratings_df['rating'] = dd.to_numeric(tmdb_ratings_df['rating'], errors='coerce')
tmdb_ratings_df['timestamp'] = dd.to_datetime(tmdb_ratings_df['timestamp'], errors='coerce')

netflix_ratings_df['CustId'] = dd.to_numeric(netflix_ratings_df['CustId'], errors='coerce')
netflix_ratings_df['Rating'] = dd.to_numeric(netflix_ratings_df['Rating'], errors='coerce')
netflix_ratings_df['Date'] = dd.to_datetime(netflix_ratings_df['Date'], errors='coerce')

rotten_movies_df['audienceScore'] = dd.to_numeric(rotten_movies_df['audienceScore'], errors='coerce')
rotten_movies_df['tomatoMeter'] = dd.to_numeric(rotten_movies_df['tomatoMeter'], errors='coerce')
rotten_movies_df['runtimeMinutes'] = dd.to_numeric(rotten_movies_df['runtimeMinutes'], errors='coerce')

rotten_reviews_df['scoreSentiment'] = dd.to_numeric(rotten_reviews_df['scoreSentiment'], errors='coerce')

# Normalize movie titles to lowercase for consistent merging
tmdb_movies_df['title'] = tmdb_movies_df['title'].str.lower().str.strip()
movielens_movies_df['title'] = movielens_movies_df['title'].str.lower().str.strip()
imdb_movies_df['primaryTitle'] = imdb_movies_df['primaryTitle'].str.lower().str.strip()
rotten_movies_df['title'] = rotten_movies_df['title'].str.lower().str.strip()

# Drop rows with missing critical information
tmdb_movies_df = tmdb_movies_df.dropna(subset=['title'])
tmdb_ratings_df = tmdb_ratings_df.dropna(subset=['userId', 'tmdbId', 'rating'])
netflix_ratings_df = netflix_ratings_df.dropna(subset=['CustId', 'Rating', 'MovieId'])
movielens_movies_df = movielens_movies_df.dropna(subset=['title'])
imdb_movies_df = imdb_movies_df.dropna(subset=['primaryTitle'])
rotten_movies_df = rotten_movies_df.dropna(subset=['title'])

# Merge Netflix ratings with MovieLens movies to map movie IDs to titles
netflix_merged_df = dd.merge(netflix_ratings_df, movielens_movies_df, left_on='MovieId', right_on='movieId', how='inner')

# Merge TMDB movies with TMDB ratings using 'tmdbId'
tmdb_merged_df = dd.merge(tmdb_movies_df, tmdb_ratings_df, left_on='tmdbId', right_on='tmdbId', how='inner')

# Merge TMDB with IMDB on movie titles
merged_df = dd.merge(tmdb_merged_df, imdb_movies_df, left_on='title', right_on='primaryTitle', how='inner')

# Merge the resulting data with MovieLens movies on titles
merged_df = dd.merge(merged_df, movielens_movies_df, left_on='title', right_on='title', how='inner')

# Merge the resulting data with Rotten Tomatoes on titles
final_merged_df = dd.merge(merged_df, rotten_movies_df, left_on='title', right_on='title', how='left')

# Compute and export the final merged DataFrame to CSV in chunks
final_merged_df.to_csv('final_merged_data.csv', index=False, single_file=True)

print("Final merged data exported to CSV.")

Final merged data exported to CSV.
